In [9]:
from enum import Enum


class OutlierMethod(Enum):
    TRIM = "trim"
    WINSER = "winser"

In [17]:
def test(method: OutlierMethod):
    if method == OutlierMethod.TRIM:
        print("trim")
    else:
        print("wins")

In [18]:
test(method=OutlierMethod.TRIM)

OutlierMethod.TRIM
